In [1]:
#python -m ipykernel install --user --name astrodino --display-name "Python (astrodino)"
import sys
print(sys.executable)   # current python folder
print(sys.version)      # Python ver

import site
print(site.getsitepackages())

!which python

/u/yacheng/conda-envs/astrodino/bin/python
3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:29:10) [GCC 14.3.0]
['/u/yacheng/conda-envs/astrodino/lib/python3.10/site-packages']
/mpcdf/soft/SLE_15/packages/x86_64/jupyterlab/3.6.3/bin/python


In [2]:
import dinov2

In [3]:
import os, sys

sys.path.append("../..")

import torch
import numpy as np
from astropy.table import Table

from astroclip.env import format_with_env
from morphology_utils.models import train_eval_on_question
from morphology_utils.plotting import plot_radar

ASTROCLIP_ROOT = format_with_env("{ASTROCLIP_ROOT}")

# Load the data
galaxy_zoo = Table.read(f"/ptmp/yacheng/outthere_ssl/images/galaxy_zoo/galaxy_zoo_embeddings.hdf5")

embedding_keys = [key for key in galaxy_zoo.keys() if 'embeddings' in key]
print("Embedding keys:", embedding_keys)

'''
# use unique names
X = {
    "AstroCLIP": torch.tensor(galaxy_zoo["astroclip_embeddings"]),
    "AstroDINO": torch.tensor(galaxy_zoo["astrodino_embeddings"]),
    "Stein": torch.tensor(galaxy_zoo["stein_embeddings"]),
}
'''

#use unique column embedding names from embedding_keys
X = {
    key: torch.tensor(np.array(galaxy_zoo[key]))
    for key in embedding_keys
}


# Get the names of the columns
names = names = [
    "smooth",
    "disk-edge-on",
    "spiral-arms",
    "bar",
    "bulge-size",
    "how-rounded",
    "edge-on-bulge",
    "spiral-winding",
    "spiral-arm-count",
    "merging",
]

# Get the labels
galaxy_zoo.remove_columns(
    embedding_keys
)
classifications = galaxy_zoo
    
# Get the key list
keys = {
    name: {
        "target": [
            key
            for key in classifications.colnames
            if name in key and "debiased" in key and "mask" not in key
        ],
        "counts": [
            key
            for key in classifications.colnames
            if name in key and "total-votes" in key
        ][0],
    }
    for name in names
}

/u/yacheng/conda-envs/astrodino/lib/python3.10/site-packages/lightning/fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
/u/yacheng/conda-envs/astrodino/lib/python3.10/site-packages/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/u/yacheng/conda-envs/astrodino/lib/python3.10/site-packages/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/u/yacheng/conda-envs/astrodino/lib/python3.10/site-packages/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


Embedding keys: ['astrodino_3z_7e5_vit_base_embeddings', 'astrodino_3z_7e5_vit_large_embeddings']


In [5]:
galaxy_zoo

iauname,ra,dec,redshift,elpetro_absmag_r,elpetro_absmag_r.mask,sersic_nmgy_r,petro_th50,petro_th90,petro_theta,upload_group,active_learning_on,in_gzd_ab,png_loc,smooth-or-featured_total-votes,smooth-or-featured_smooth,smooth-or-featured_smooth_fraction,smooth-or-featured_smooth_debiased,smooth-or-featured_smooth_debiased.mask,smooth-or-featured_featured-or-disk,smooth-or-featured_featured-or-disk_fraction,smooth-or-featured_featured-or-disk_debiased,smooth-or-featured_featured-or-disk_debiased.mask,smooth-or-featured_artifact,smooth-or-featured_artifact_fraction,smooth-or-featured_artifact_debiased,smooth-or-featured_artifact_debiased.mask,disk-edge-on_total-votes,disk-edge-on_yes,disk-edge-on_yes_fraction,disk-edge-on_yes_fraction.mask,disk-edge-on_yes_debiased,disk-edge-on_yes_debiased.mask,disk-edge-on_no,disk-edge-on_no_fraction,disk-edge-on_no_fraction.mask,disk-edge-on_no_debiased,disk-edge-on_no_debiased.mask,has-spiral-arms_total-votes,has-spiral-arms_yes,has-spiral-arms_yes_fraction,has-spiral-arms_yes_fraction.mask,has-spiral-arms_yes_debiased,has-spiral-arms_yes_debiased.mask,has-spiral-arms_no,has-spiral-arms_no_fraction,has-spiral-arms_no_fraction.mask,has-spiral-arms_no_debiased,has-spiral-arms_no_debiased.mask,bar_total-votes,bar_strong,bar_strong_fraction,bar_strong_fraction.mask,bar_strong_debiased,bar_strong_debiased.mask,bar_weak,bar_weak_fraction,bar_weak_fraction.mask,bar_weak_debiased,bar_weak_debiased.mask,bar_no,bar_no_fraction,bar_no_fraction.mask,bar_no_debiased,bar_no_debiased.mask,bulge-size_total-votes,bulge-size_dominant,bulge-size_dominant_fraction,bulge-size_dominant_fraction.mask,bulge-size_dominant_debiased,bulge-size_dominant_debiased.mask,bulge-size_large,bulge-size_large_fraction,bulge-size_large_fraction.mask,bulge-size_large_debiased,bulge-size_large_debiased.mask,bulge-size_moderate,bulge-size_moderate_fraction,bulge-size_moderate_fraction.mask,bulge-size_moderate_debiased,bulge-size_moderate_debiased.mask,bulge-size_small,bulge-size_small_fraction,bulge-size_small_fraction.mask,bulge-size_small_debiased,bulge-size_small_debiased.mask,bulge-size_none,bulge-size_none_fraction,bulge-size_none_fraction.mask,bulge-size_none_debiased,bulge-size_none_debiased.mask,how-rounded_total-votes,how-rounded_round,how-rounded_round_fraction,how-rounded_round_fraction.mask,how-rounded_round_debiased,how-rounded_round_debiased.mask,how-rounded_in-between,how-rounded_in-between_fraction,how-rounded_in-between_fraction.mask,how-rounded_in-between_debiased,how-rounded_in-between_debiased.mask,how-rounded_cigar-shaped,how-rounded_cigar-shaped_fraction,how-rounded_cigar-shaped_fraction.mask,how-rounded_cigar-shaped_debiased,how-rounded_cigar-shaped_debiased.mask,edge-on-bulge_total-votes,edge-on-bulge_boxy,edge-on-bulge_boxy_fraction,edge-on-bulge_boxy_fraction.mask,edge-on-bulge_boxy_debiased,edge-on-bulge_boxy_debiased.mask,edge-on-bulge_none,edge-on-bulge_none_fraction,edge-on-bulge_none_fraction.mask,edge-on-bulge_none_debiased,edge-on-bulge_none_debiased.mask,edge-on-bulge_rounded,edge-on-bulge_rounded_fraction,edge-on-bulge_rounded_fraction.mask,edge-on-bulge_rounded_debiased,edge-on-bulge_rounded_debiased.mask,spiral-winding_total-votes,spiral-winding_tight,spiral-winding_tight_fraction,spiral-winding_tight_fraction.mask,spiral-winding_tight_debiased,spiral-winding_tight_debiased.mask,spiral-winding_medium,spiral-winding_medium_fraction,spiral-winding_medium_fraction.mask,spiral-winding_medium_debiased,spiral-winding_medium_debiased.mask,spiral-winding_loose,spiral-winding_loose_fraction,spiral-winding_loose_fraction.mask,spiral-winding_loose_debiased,spiral-winding_loose_debiased.mask,spiral-arm-count_total-votes,spiral-arm-count_1,spiral-arm-count_1_fraction,spiral-arm-count_1_fraction.mask,spiral-arm-count_1_debiased,spiral-arm-count_1_debiased.mask,spiral-arm-count_2,spiral-arm-count_2_fraction,spiral-arm-count_2_fraction.mask,spiral-arm-count_2_debiased,spiral-arm-count_2_debiased.mask,spiral-arm-

In [6]:
# Select first 80% for train and last 20% for test
train_indices = int(0.9 * len(classifications))

X_train, X_test = {}, {}
for key in X.keys():
    X_train[key] = X[key][:train_indices]
    X_test[key] = X[key][train_indices:]

classifications_train, classifications_test = (
    classifications[:train_indices],
    classifications[train_indices:],
)

In [7]:
# This is the total number of possible votes
total_counts_train = classifications_train[keys["smooth"]["counts"]].data

# Get accuracy and F1 score on each question
outputs = {key: {} for key in X.keys()}
for name in names:
    question, num_classes = name, len(keys[name]["target"])

    # Get the train samples above 50% answered
    counts_train = classifications_train[keys[name]["counts"]].data
    #train_mask = np.where(counts_train / total_counts_train > 0.5)[0]
    train_mask = [True] * len(counts_train)

    # Get the test samples above 34 answers
    counts_test = classifications_test[keys[name]["counts"]].data
    test_mask = np.where(counts_test > 34)[0]
    #test_mask = [True] * len(counts_test)

    # Get train and test
    y_train = torch.tensor(
        classifications_train[keys[name]["target"]].to_pandas().values
    )[train_mask]
    y_test = torch.tensor(
        classifications_test[keys[name]["target"]].to_pandas().values
    )[test_mask]

    train_nan_mask = torch.isnan(y_train).any(axis=1)
    test_nan_mask = torch.isnan(y_test).any(axis=1)

    # Train and evaluate on each model
    print(f"Training on question: {question}...")
    for model in X.keys():
        X_train_local = X_train[model][train_mask][~train_nan_mask]
        X_test_local = X_test[model][test_mask][~test_nan_mask]
        outputs[model][name] = train_eval_on_question(
            X_train_local,
            X_test_local,
            y_train,
            y_test,
            X_train_local.shape[1],
            num_classes=num_classes,
            MLP_dim=256,
            epochs=25,
            dropout=0.2,
        )
        print(
            f"Model: {model}, Accuracy: {outputs[model][name]['Accuracy']:.4f}, F1: {outputs[model][name]['F1 Score']:.4f}"
        )
    print("Done!")

Training on question: smooth...
Model: astrodino_3z_7e5_vit_base_embeddings, Accuracy: 0.7889, F1: 0.6961
Model: astrodino_3z_7e5_vit_large_embeddings, Accuracy: 0.7889, F1: 0.6961
Done!
Training on question: disk-edge-on...
Model: astrodino_3z_7e5_vit_base_embeddings, Accuracy: 0.9393, F1: 0.9321
Model: astrodino_3z_7e5_vit_large_embeddings, Accuracy: 0.8850, F1: 0.8310
Done!
Training on question: spiral-arms...
Model: astrodino_3z_7e5_vit_base_embeddings, Accuracy: 0.8619, F1: 0.8987
Model: astrodino_3z_7e5_vit_large_embeddings, Accuracy: 0.8536, F1: 0.8940
Done!
Training on question: bar...
Model: astrodino_3z_7e5_vit_base_embeddings, Accuracy: 0.5230, F1: 0.3562
Model: astrodino_3z_7e5_vit_large_embeddings, Accuracy: 0.5230, F1: 0.3562
Done!
Training on question: bulge-size...
Model: astrodino_3z_7e5_vit_base_embeddings, Accuracy: 0.7280, F1: 0.7184
Model: astrodino_3z_7e5_vit_large_embeddings, Accuracy: 0.7741, F1: 0.7681
Done!
Training on question: how-rounded...
Model: astrodino

In [8]:
outputs.keys()

dict_keys(['astrodino_3z_7e5_vit_base_embeddings', 'astrodino_3z_7e5_vit_large_embeddings'])

In [9]:
# Clean up labels: remove substrings from keys
REMOVE_SUBSTRINGS = ["_3z_band_stable", "_embeddings"]

def _strip_name(name: str) -> str:
    out = name
    for s in REMOVE_SUBSTRINGS:
        out = out.replace(s, "")
    # collapse consecutive underscores
    while "__" in out:
        out = out.replace("__", "_")
    return out.strip("_")

# Rename all keys in outputs dict
outputs = { _strip_name(k): v for k, v in outputs.items() }
print("Renamed output keys:", list(outputs.keys()))

# (Optional) map to prettier display labels if desired
# label_map = {
#     "astrodino": "AstroDINO Base",
#     "astrodino_multi_epochs": "AstroDINO Multi Epochs",
#     "astrodino_multi_epochs_vit_large": "AstroDINO Large",
# }
# outputs = { label_map.get(k, k): v for k, v in outputs.items() }

# Plot radar plots
plot_radar(outputs, metric="Accuracy", file_path=f"./outputs/radar_accuracy.png")
plot_radar(outputs, metric="F1 Score", file_path=f"./outputs/radar_f1_score.png")

Renamed output keys: ['astrodino_3z_7e5_vit_base', 'astrodino_3z_7e5_vit_large']


In [78]:
outputs.keys()

dict_keys(['astrodino', 'astrodino_multi_epochs', 'astrodino_multi_epochs_vit_large'])